## Import Libraries 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
#!/usr/bin/env python
from __future__ import print_function
import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from time import time
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [3]:
print(tf.__version__)
print(keras.__version__)

1.14.0
2.2.4-tf


## Load Model 

In [5]:
model = keras.models.load_model('model_19-0.00.h5')

/home/khan74/.conda/envs/gwsurr/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:891: UserWarning: training_utils is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


## Load Test Data 

In [6]:
file_path = '/home/khan74/project/SMR_BBM/data/test.hdf5'
f = h5py.File(file_path, 'r')
keys = list(f.keys())
dset = f[keys[0]] # waveforms
lset = f[keys[1]] # Ground Truth labels: q, s1, s2

data = dset[:100]  # load first ten samlples
data = data.reshape([data.shape[0], data.shape[1], 1])

f.close()

## Predict 

In [7]:
p_q, p_ap = model.predict(data, batch_size=2)

In [8]:
predictions = np.hstack([p_q, p_ap])

#### Convert to dataframe 

In [9]:
predictions_df = pd.DataFrame(data=predictions,
                             columns=['q', 's1', 's2', 'chi', 'S_eff'])

Were:

- q is mass ratio
- s1 is spin of primary
- s2 is spin of secondary


- chi is $$ \chi_{eff} = \frac{m_1s_1 + m_2 s_2}{m_1 + m_2} =  \frac{ q s_1 + s_2 }{q + 1} $$


- S_eff is $$ S^{eff} = \sigma_1 s_1 + \sigma_2 s_2 $$

        - where       
$$ \sigma_1 = 1 + \frac{3}{4} \frac{m_2}{m_1} =  1 + \frac{3}{4q} $$  
        - and          
$$ \sigma_2 = 1 + \frac{3}{4} \frac{m_1}{m_2} =  1 + \frac{3q}{4} $$

In [10]:
predictions_df.head()

,q,s1,s2,chi,S_eff
0,1.336917,-0.871400,-0.677907,-0.765067,-2.894305
1,1.165531,-0.837524,-0.714918,-0.767969,-2.924671
2,1.053830,-0.872219,-0.637561,-0.763909,-2.754155
3,1.063442,-0.749734,-0.713964,-0.732895,-2.780643
4,1.096151,-0.778480,-0.734360,-0.748120,-2.699069
